In [52]:
import pandas as pd
import numpy as np
import collections,numpy


#读取数据
df = pd.read_csv('./data_process/all_features.csv')
# print(df)

#统计username的重复次数，也就相当于统计每个学生选了几门课
count = pd.value_counts(df['username'])
#将count转化为DataFrame类型
count = pd.DataFrame(count)
#为列重命名
count.rename(columns={'username': 'count'}, inplace=True)
#设置索引列
count.index.name = 'username'

#将两个按照username列进行合并
end = pd.merge(df, count, on='username')

#按照count列进行排序
end = end.sort_values(by='count', ascending=False)

end = end.query('count > 4')

end = end.drop('count',axis = 1)

end.to_csv('./data_process/count_course_number.csv',index = False)

In [54]:
activity_data = pd.read_csv('./data_process/count_course_number.csv',
                            index_col='enroll_id'
                            )

uid_list=set(activity_data['username'])
cid_list=set(activity_data['course_id'])
uid_old_new=dict(zip(uid_list,range(0,len(uid_list))))
cid_old_new=dict(zip(cid_list,range(0,len(cid_list))))

activity_data['username']=activity_data['username'].map(uid_old_new)
activity_data['course_id']=activity_data['course_id'].map(cid_old_new)




activity_data.to_csv('./data_process/number_data.csv')
activity_data

,Unnamed: 0,all#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,create_thread#num,create_comment#num,delete_thread#num,delete_comment#num,close_forum#num,click_courseware#num,close_courseware#num,username,course_id,truth
enroll_id,,,,,,,,,,,,,,,,,,,,,
166191,20366,137,0,0,22,0,17,18,11,0,...,0,0,0,0,0,31,14,9321,92,1
167060,20987,12,0,0,0,0,0,3,1,0,...,0,0,0,0,0,2,2,9321,52,1
367438,124105,95,0,0,0,0,2,18,12,0,...,0,0,0,0,0,20,12,9321,29,0
363206,122347,203,0,0,0,0,1,26,80,0,...,0,0,0,0,0,0,16,9321,58,1
352893,119219,29,0,0,3,0,3,1,2,0,...,0,0,0,0,0,2,1,9321,184,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29854,10636,669,26,110,109,36,42,72,58,1,...,0,13,0,0,0,62,47,6364,46,0
400088,134160,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8639,3,1
365512,123302,90,1,13,12,8,9,4,4,0,...,0,0,0,0,0,14,14,6364,29,0


In [55]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

#读取量化的参与意愿
concat_data = pd.read_csv('./row_data/concat_data.csv',usecols=[1,2])

#读取最初的
data = pd.read_csv('./data_process/number_data.csv',
                   usecols=[0,22,23,24],
                   )


activity = pd.merge(data, concat_data, on='enroll_id', how='inner')
# del activity['Unnamed: 0']
activity = activity.rename(columns={'pred': 'complete'})
activity


,enroll_id,username,course_id,truth,complete
0,166191,9321,92,1,0.951944
1,167060,9321,52,1,0.603655
2,367438,9321,29,0,0.931955
3,363206,9321,58,1,0.053649
4,352893,9321,184,1,0.961164
...,...,...,...,...,...
76939,29854,6364,46,0,0.976490
76940,400088,8639,3,1,0.136594
76941,365512,6364,29,0,0.216140
76942,275788,7340,93,1,0.927186


In [56]:
del activity['enroll_id']
activity
activity.to_csv('./data_process/model_input_willingness.csv',index=0)

In [57]:
activity

,username,course_id,truth,complete
0,9321,92,1,0.951944
1,9321,52,1,0.603655
2,9321,29,0,0.931955
3,9321,58,1,0.053649
4,9321,184,1,0.961164
...,...,...,...,...
76939,6364,46,0,0.976490
76940,8639,3,1,0.136594
76941,6364,29,0,0.216140
76942,7340,93,1,0.927186


### 划分训练集和测试集

In [58]:
#只按照78：2的比例去划分

import random
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import KFold
# 定义DataSet类
class DataSet():
    # 初始化函数，self指向的是实例化对象
    def __init__(self, basedir):
        self.basedir = basedir
        read_dir = basedir + '/data_process/'
        save_dir = basedir + '/data_process/'
        data = pd.read_csv(read_dir + "model_input_willingness.csv")
        # 下方划分数据集的时候需要调用self.record，将uid设置为索引号，
        # 返回的record是一个以uid为索引的dataframe
        self.record = data.set_index('username')

        # 将record重点索引去重后，以列表形式保存
        self.total_stu_list = list(set(self.record.index))
        self.save_dir = save_dir

    def get_train_test(self, test_ratio = 0.2):
            train, test= split_record(self.record, test_ratio= test_ratio)
            return train, test  # 这里划分了， index_loader里的id应该从这里取


def split_record(record, test_ratio = 0.2):
    total_stu_list = list(set(record.index))
    train_data = [[], [], [], []]
    test_data = [[], [], [], []]
    for stu in total_stu_list:
        # 遍历这个学生的所有列
        stu_data = record.loc[stu, :]
        # 将cid、truth、以及activity这三列分别定义为数组
        stu_cour = np.array(stu_data['course_id'])
        stu_truth = np.array((stu_data['truth']))
        stu_activity = np.array(stu_data['complete'])
        
      
        length = len(stu_cour)

        
#         length = len(stu_cour)
        index_list = list(range(length))
 
        test_index = random.sample(index_list, int(length * test_ratio))
        #test_index = random.sample(index_list, int(length * test_ratio))

        #剩下的所有为训练集
        train_index = list(set(index_list) - set(test_index))

        train_data[0].extend([stu] * len(train_index))
        train_data[1].extend(stu_cour[train_index])
        train_data[2].extend(stu_truth[train_index])
        train_data[3].extend(stu_activity[train_index])


        test_data[0].extend([stu] * len(test_index))
        test_data[1].extend(stu_cour[test_index])
        test_data[2].extend(stu_truth[test_index])
        test_data[3].extend(stu_activity[test_index])

    train = pd.DataFrame({'username': train_data[0], 'course_id': train_data[1],'truth': train_data[2],'complete': train_data[3]},).set_index('username')
    test = pd.DataFrame({'username': test_data[0], 'course_id': test_data[1], 'truth': test_data[2],'complete': test_data[3]},).set_index('username')
    return train, test

if __name__ == '__main__':
    # ----------基本参数--------------
    basedir = 'D:/pythonProject/AAAI'
    dataSet = DataSet(basedir)
    save_dir = dataSet.save_dir
    # 这里ratio是测试集的个数，用来调参使用，使其值分别为2，3，4
    test_ratio = 0.2
    train_data, test_data = dataSet.get_train_test(test_ratio= test_ratio)
    print(len(train_data))
    print(len(test_data))
    train_data.to_csv(save_dir+'train_data_2.csv')
    test_data.to_csv(save_dir+'test_data_2.csv')
    # ----------基本参数--------------



63880
13064


In [47]:
train_data = pd.read_csv('./data_process/train_data_2.csv')
test_data = pd.read_csv('./data_process/test_data_2.csv')

del train_data['truth']
del test_data['truth']

train_data.to_csv('./model_input_data/model_train_data_2.csv',index=0)
test_data.to_csv('./model_input_data/model_test_data_2.csv',index=0)



In [48]:
import pandas as pd
train_data = pd.read_csv('./model_input_data/model_train_data_2.csv')
train_data


,username,course_id,complete
0,0,114,0.914391
1,0,208,0.035315
2,0,45,0.929883
3,0,66,0.647070
4,0,97,0.931246
...,...,...,...
58218,9439,89,0.901017
58219,9440,2,0.944082
58220,9440,162,0.846969
58221,9440,139,0.950144


In [49]:
train_data['complete'] = train_data['complete'].map(lambda x: 1-x)
train_data

,username,course_id,complete
0,0,114,0.085609
1,0,208,0.964685
2,0,45,0.070117
3,0,66,0.352930
4,0,97,0.068754
...,...,...,...
58218,9439,89,0.098983
58219,9440,2,0.055918
58220,9440,162,0.153031
58221,9440,139,0.049856


In [50]:
test_data = pd.read_csv('./model_input_data/model_test_data_2.csv')
test_data

,username,course_id,complete
0,0,84,0.884160
1,0,155,0.351657
2,0,111,0.335319
3,1,226,0.917891
4,1,102,0.953184
...,...,...,...
18716,9437,213,0.345646
18717,9438,224,0.913971
18718,9438,55,0.744407
18719,9439,8,0.665413


In [51]:
test_data['complete'] = test_data['complete'].map(lambda x: 1-x)
test_data

,username,course_id,complete
0,0,84,0.115840
1,0,155,0.648343
2,0,111,0.664681
3,1,226,0.082109
4,1,102,0.046816
...,...,...,...
18716,9437,213,0.654354
18717,9438,224,0.086029
18718,9438,55,0.255593
18719,9439,8,0.334587


In [15]:
train_data.to_csv('./model_input_data/model_train_data.csv')
test_data.to_csv('./model_input_data/model_test_data.csv')

### 邻接矩阵处理

In [16]:
adj2_data = pd.read_csv('./model_input_data/model_train_data.csv',usecols = [1,2])
df = pd.crosstab(adj2_data.username,adj2_data.course_id)
df.to_csv('./data_process/adjacency_first.csv',)

In [ ]:
import pandas as pd
train_data_weight = pd.read_csv('./data_process/train_data_2.csv',usecols=[2,3,4])
train_data_weight

In [ ]:
adjacency = pd.read_csv('./data_process/adjacency_first.csv',index_col = 0)
adjacency

In [ ]:
#遍历每行每列，将模型最后输出的结果由矩阵形式转化为三列
row_index = adjacency.index.values
col_index = adjacency.columns.values
aw = []
for row in row_index:
    for col in col_index:
        adjacency_weight = adjacency.loc[row,col]
        #将遍历到的每行每列的数值保存在result这个全局变量里面，方便后面的保存
        aw.append([row,col,adjacency_weight])
aw = pd.DataFrame(aw)
aw.columns=['username', 'course_id', 'adjacency_weight']
aw

In [ ]:
aw.to_csv('./data_process/aw.csv',index = 'username')
import pandas as pd
aw = pd.read_csv('./data_process/aw.csv',index_col = 0)
aw

In [ ]:
adjacency_w = pd.merge(train_data_weight,aw,on=['username','course_id'], how='outer')
adjacency_w = adjacency_w.fillna(-1)
adjacency_w

In [ ]:
adjacency_w['weight'] = 0
for id, row in enumerate(adjacency_w.itertuples()):
    truth = getattr(row, 'truth')
    #规定活跃度值大于mean值则为不辍学，若小于mean则记为辍学
    if truth ==0:
          adjacency_w['weight'][id] = 1
    if truth ==1:
        adjacency_w['weight'][id] = 0.1
    if truth ==-1:
        adjacency_w['weight'][id] = 0
adjacency_w
    

In [ ]:
adj = adjacency_w.pivot(index='username', columns='course_id', values='weight')
adj.to_csv('./model_input_data/adjacency_weight.csv',index = 'username')

### 课程聚合

In [17]:
action_train = pd.read_csv('./data_process/number_data.csv')
data = pd.merge(action_train, adj2_data, on=['username','course_id'], how='inner')
del data['enroll_id']
del data['Unnamed: 0']
data

,all#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,reset_problem#num,...,create_thread#num,create_comment#num,delete_thread#num,delete_comment#num,close_forum#num,click_courseware#num,close_courseware#num,username,course_id,truth
0,137,0,0,22,0,17,18,11,0,0,...,0,0,0,0,0,31,14,9321,146,1
1,12,0,0,0,0,0,3,1,0,0,...,0,0,0,0,0,2,2,9321,79,1
2,95,0,0,0,0,2,18,12,0,2,...,0,0,0,0,0,20,12,9321,191,0
3,203,0,0,0,0,1,26,80,0,0,...,0,0,0,0,0,0,16,9321,124,1
4,29,0,0,3,0,3,1,2,0,0,...,0,0,0,0,0,2,1,9321,128,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58218,317,30,67,41,33,64,9,4,0,0,...,0,1,0,0,0,0,64,6589,183,0
58219,669,26,110,109,36,42,72,58,1,0,...,0,13,0,0,0,62,47,6364,92,0
58220,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8639,61,1
58221,17,0,1,1,1,3,0,0,0,0,...,0,0,0,0,0,6,0,7340,17,1


In [18]:
data.to_csv('./data_process/all_train_data.csv')

In [19]:

import csv

#生成字典，key是课程名，value是课程所对应的每一行数据
result = {}
if __name__ == '__main__':
    #定义列表第一行
    switch = 0
    #用header来记录表头
    header = []
    #打开文件
    with open("./data_process/all_train_data.csv") as f:
        #读取结果是列表
        csv_reader = csv.reader(f)
        # next(csv_reader)

        #循环遍历每个文件的每一行
        for row in csv_reader:
            #如果遍历的是第一行，则将其值赋值给表头
            if switch == 0:
                header = row
                #然后continue再来一次遍历，switch变成1之后运行后面的程序
                switch = 1
                continue

            # 21列是用户ID，如果学生ID没有在字典里，则开辟空间，将header和row存进去，如果有，则把内容row拼接在后面
            if row[21] not in result.keys():
                result[row[21]] = []
                result[row[21]].append(header)
                result[row[21]].append(row)
            else:
                result[row[21]].append(row)
        f.close()

    for file_name in result.keys():

        #print(file_name)
        with open("./sort_user/"+file_name+".csv", 'w', newline="") as f:
            writer = csv.writer(f)
            writer.writerows(result[file_name])
            # for row in result[file_name]:
            #     writer.writerow(row)
            f.close()




In [20]:
import pandas as pd
import glob
import csv
import numpy as np

input_path = './sort_user/*.csv'
# glob.glob()函数将会匹配给定路径下的所有pattern，并以列表形式返回
all_files = glob.glob(input_path)

# 循环遍历每个文件
for file in all_files:
    data_frame = pd.read_csv(file,
                             index_col=None,
                             # usecols=[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                             encoding='gbk')

    # 拿到目标列的Index值，通过Index对选中列求平均值
    key_cols = data_frame.keys().tolist()[2:21]
    # apply不能直接改变列值，需要更新df数据
    data_frame[key_cols] = data_frame[key_cols].apply(
        lambda x: x.mean()
    )

    # 导出
    file_name = file.split('\\')[-1]  # spilt分开字符串，把文件名拿出来
    # 将数据求平均值后导出到另外一个文件夹，以原文件名称 取消第一列的索引
    data_frame.to_csv('./mean_user/' + file_name, index=False)


In [21]:
import pandas as pd
import glob
import csv
import numpy as np

input_path = './mean_user/*.csv'

# glob.glob()函数将会匹配给定路径下的所有pattern，并以列表形式返回
all_files = glob.glob(input_path)
all_data = pd.DataFrame()

# 循环遍历每个文件
for file in all_files:
    data_frame = pd.read_csv(file,
                             usecols=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
                             index_col=None,
                             encoding='gbk')
    data = data_frame.iloc[0]

    all_data = all_data.append(data)
all_data.to_csv('./model_input_data/avg_course.csv',index='username')
   

C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: Fut

C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: Fut

C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: Fut

C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: Fut

C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: Fut

...

C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: Fut

C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: Fut

C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: Fut

C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: Fut

C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
C:\Users\lishu\AppData\Local\Temp/ipykernel_6696/43513757.py:20: Fut

### tensor处理

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
import numpy as np

In [ ]:
#读取最原始的数据，这里对应的是Data
row_data = pd.read_csv('./data_process/all_train_data.csv')
row_data

In [ ]:
#建立一个空的tensor，保存Data的数据
Data  = torch.empty(9441,247,19)

In [ ]:
actions = row_data.columns.values[2:21]
actions

In [ ]:
for index, row in tqdm(row_data.iterrows(),total=row_data.shape[0]):
    uid = row['username']
    cid = row['course_id']
#     print(f'uid {uid} cid {cid} index {index}')
    for action_index, action in enumerate(actions):
        Data[uid, cid, action_index] = row_data.query('(username==@uid) and (course_id==@cid)')[action].values[0]
torch.save(Data,'./data_process/Data.pt')

In [ ]:
Data = torch.load('./data_process/Data.pt')
Data

In [ ]:
Data_csa = Data.sum(axis = 2)
Data_csa.size()

In [ ]:
Data_csa_1 = Data_csa.unsqueeze(2)
Data_csa_1.size()

In [ ]:
Data_csa_end = Data_csa_1.repeat(1,1,19)
Data_csa_end.size()

In [ ]:
four_data = Data/Data_csa_end
four_data.size()

In [ ]:
Data_pbk1_end = np.nan_to_num(four_data)

In [ ]:
Data_pbk1_end = torch.from_numpy(Data_pbk1_end)
Data_pbk1_end

In [ ]:
torch.save(Data_pbk1_end,'./model_input_data/data_csa.pt')